<a href="https://colab.research.google.com/github/Agithos/Anakthsh/blob/main/Anakthsh_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ΛΕΙΤΟΥΡΓΙΑ SCRIPT
---
###Βήματα λειτουργίας
* Ctrl+F9 (Run All cells)
* Copy-paste σε Browser το URL του ngrok web-server
  * Αυτό που γράφει δηλαδή "Running on *****.ngrok.io"
  * Σε Chrome πρέπει να είναι σε Private mode. Αλλιώς, Mozilla
---
###Επιλογές συγκεκριμένης λύσης
* Ανακτώ από τα πρώτα 60 links για οικονομία χρόνου.  (δηλαδή με anakthsh(links,60) )
* Μπορώ να ανακτήσω από όλα τα links αν αλλάξω σε anakthsh(links,0)

#### Λογική ανάκτησης ερωτήσεων
* Όλη η ερώτηση βρίσκεται μέσα σε \<strong>
  * Αν υπάρχει μέρος που δεν είναι μέσα σε \<strong>, θα είναι bold μέρος μιας απάντησης
* Τελευταίο char της ερώτησης, αφού έχω καθαρίσει με .strip() είναι '?'

#### Ποια questions έχει ανακτήσει
* li μέσα σε Order list
* p με \<strong> να κάνει wrap την Ερώτηση
  * Ή διαδοχικά \<strong>
* h3 για κάθε ερώτηση

---
#### Παραδείγματα Queries
* Act of Europe?
* Economy of Europe 
* Ukraine actions


####Πιθανά προβλήματα
* Φαίνονται στο τέλος του Notebook. Πιο πιθανά όταν σταματάει και ξαναξεκινάει ο ngrok-server
---


### Pip installs και κατεβασμα elastic

In [ ]:
%%shell
apt install chromium-chromedriver
pip install beautifulsoup4
pip install selenium
pip install farm-haystack

pip install pyngrok==4.1.1
pip install flask-ngrok

git clone "https://github.com/Agithos/Anakthsh.git"

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

# rm -rf Anakthsh
# rm -rf elasticsearch-7.9.2/

In [ ]:
ngrokToken = "" #@param {type:"string"}
!ngrok authtoken $ngrokToken

###Init haystack, elastic server και selenium browser

In [ ]:
## INIT ##
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
import math
import os
from subprocess import Popen,PIPE,STDOUT

from haystack.nodes import EmbeddingRetriever
from haystack.nodes import ElasticsearchRetriever
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.document_stores import elasticsearch_index_to_document_store
from haystack.pipelines import FAQPipeline

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)
)
!sleep 30

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome('chromedriver',options=chrome_options)

baseURL = "https://ec.europa.eu/commission/presscorner/"
startURL = "https://ec.europa.eu/commission/presscorner/advancedsearch/en?keywords=&dotyp=852&parea=0&pareaType=0&datepickerbefore=&datebefore=&commissioner=&datepickerafter=&dateafter=&pagenumber="
links = []
pattern = "<\/question>\s*(?:<\/\w*>\s*)*([\s\S]*?)\s*(?:<\w*>\s*)*<question[\s\S]*?>"

document_store = ElasticsearchDocumentStore(
  username="",
  password="",
  similarity="cosine",
  embedding_field="question_emb",
  index="document",
  embedding_dim=384)

model = "deepset/roberta-base-squad2"

retrieverElastic = ElasticsearchRetriever(document_store)
retrieverEmbedding = EmbeddingRetriever(
  document_store=document_store,
  embedding_model="sentence-transformers/all-MiniLM-L6-v2",
  use_gpu=True)

pipeEmbedding = FAQPipeline(retriever = retrieverEmbedding)
pipeElastic = FAQPipeline(retriever = retrieverElastic)


### Functions 

In [ ]:
## FUNCTIONS ##

def anakthsh(links, linkNo):
  pagesLimit = math.ceil(linkNo/10)
  if pagesLimit==1:       # to elaxisto i findAllPageLinks katevazei 2 selides
    pagesLimit = 2
  findAllPageLinks(links, pagesLimit)
  if (linkNo == 0):
    downloadLinkInfo(links)
  else:
    downloadLinkInfo(links[:linkNo])

# Soup gia opoio url exw
def makeSoup (url):
    browser.get(url)
    time.sleep(5)
    soup = BeautifulSoup(browser.page_source)
    return soup

def isNextPage(soup):
  nextButton = soup.select('a[title="Go to next page"]')
  if (nextButton):
    return True
  else:
    return False

# MAIN1: Find Links
def findAllPageLinks(links, limit):
  soup = makeSoup(startURL)
  getCurrentPageLinks(soup, links)
  # oso exei nextPage
  pageNumber = 2
  while(isNextPage(soup)):
    soup = makeSoup(startURL + str(pageNumber))
    getCurrentPageLinks(soup, links)
    pageNumber +=1

    ## TEST
    if(pageNumber == limit+1):
      break;

def getCurrentPageLinks(soupList, linksQA):
    h3 = soupList.select(".ecl-list-item__link")
    for heading in h3:
        linksQA.append(baseURL + heading["href"])

# MAIN2: Kalei getQA apo ola ta links sigkekrimenou page
def downloadLinkInfo(links):
  page = 0
  for link in links:
    page+=1
    print(page)

    soupLink = makeSoup(link)
    getQA(soupLink, link)

# GET QA kai MetaInfo kathe Selidas sigkekrimenhs (r)
def getQA(soupQA, link):
  section = soupQA.select("#inline-nav-1")[0]   # select() epistrefei [] - ResultSet me tags
  findQuestions(section)
  pageInfo = getMetaInfo(soupQA)

  strippedSection = stripSection(section)
  answerMatches = re.findall(pattern, strippedSection)
  # Edw soupQA_section + "<question>"
  # print(answerMatches[-1])
  questions = soupQA.select("question")
  # print(len(answerMatches))

  if (len(questions) <= 1):
    print(link)
    print("Error: One or zero questions found")
    if (len(questions) == 0):
      return
  if len(answerMatches) != len(questions):
    print(link)
    print(len(questions))
    print(len(answerMatches))
  print()

  questionsText = []
  for question in questions:
    questionsText.append(question.get_text().strip())

  embeddingArray = makeEmbedding(questionsText, retrieverEmbedding)

  QA_Array = makeQA_Array(answerMatches, questionsText, pageInfo, embeddingArray)
  updateHaystack(document_store,QA_Array)

def stripSection(section):
  if str(section)[-16:]!="</div></section>":        #</div></section>
    print("no")
  return (str(section)[:-16] + "<question>")

def updateHaystack(document_store, QA_Array):
  document_store.write_documents(QA_Array)

# Pairnei Question, Answer, Embedding, MetaInfo apo kathena kai -> [dicts]
def makeQA_Array(answerArray, questions, pageInfo, embeddingArray):       # prosthetw metaInfo
  QA_Array = []
  index = 0
  for answer in answerArray:
    questionObject = {}
    questionObject["content"] = questions[index]
    questionObject["question_emb"] = embeddingArray[index]
    # questionObject["question_emb"] = list(map(lambda x:0, embeddingArray))
    questionObject["answer"] = answer
    questionObject["title"] = pageInfo["title"]
    questionObject["date"] = pageInfo["date"]
    questionObject["country"] = pageInfo["country"]

    QA_Array.append(questionObject)
    index += 1
  return QA_Array

# Make Embedding apo lista Questions
def makeEmbedding(questionsList, retrieverEmbedding):
  embeddingArray = retrieverEmbedding.embed_queries(texts=questionsList)
  return embeddingArray

def isNotOrderListStyle(section):
  if section.div.contents[0].name == "ol" or section.div.contents[0].name == "ul":
    firstElement = section.div.contents[0].contents[0]
    if firstElement.strip() == "":
      firstElement = firstElement.parent.contents[1]
  else:
    firstElement = section.div.contents[0]
  if isQuestion(firstElement):
    return (section.div.contents[0].name == "p")
  else:
    return True

def isQuestion(textElement):
  pText = textElement.get_text().strip()
  # print(pText)
  if textElement.parent:
    if pText == textElement.parent.get_text().strip() and len(pText)!=0 and pText[-1].strip() == "?":
      return True

    siblingNumber = len(textElement.parent.contents) -1
    siblings = []

    if siblingNumber >=1 and textElement.parent.contents[0].name=="strong":
      if textElement == textElement.parent.contents[0]:
        strongText = textElement.get_text()
        if siblingNumber <= 4:
          for i in range(siblingNumber):
            siblings.append(textElement.parent.contents[i+1])
            if textElement.parent.contents[i+1].name=="strong":
              strongText = strongText + " " + textElement.parent.contents[i+1].get_text()
          strongText.strip()
        
        if len(strongText)!=0 and strongText == textElement.parent.get_text().strip() and strongText[-1].strip()=="?":
          textElement.clear()
          textElement.append(strongText)
          for sibling in siblings:
            sibling.extract()
          # print(textElement)
          return True
  return False
def findQuestions(section):
  if (isNotOrderListStyle(section)):
    if (section.div.contents[0].name == "p"):
      qa = section.select("p strong")
    else:
      qa = section.select("h3")
      for h3 in qa:
        h3.name = "question"
        # print(h3.get_text())
      return
  else:
    qa = section.select("ol li strong")

  qNumber = 0
  for paragraph in qa:
    if isQuestion(paragraph):
      paragraph.name = "question"
      qNumber += 1
      # print(paragraph.get_text())
    else:
      continue
def getMetaInfo(soupLink):
  title = soupLink.h1.get_text()
  metaInfo = soupLink.select("span.ecl-meta__item")[1:]
  pageInfo = {"title": title, "date": metaInfo[0].get_text(), "country": metaInfo[1].get_text()}
  return pageInfo

def pipeRun(isEmbedding, query):
  if isEmbedding:
    prediction = pipeEmbedding.run(query=query, params={"Retriever":{"top_k": 5}})
  else:
    prediction = pipeElastic.run(query=query, params={"Retriever":{"top_k": 5}})
  # print(prediction["answers"])        # ["answers"][0]
  resultsList=[]
  for answer in prediction["answers"]:
    result = {}
    result["meta"] = answer.meta
    result["score"] = answer.score
    resultsList.append(result)
  resultsDict = {"answerList": resultsList, "query": prediction["query"]}
  return resultsDict



### Run program για να γεμίσουμε haystack-document_store

In [ ]:
## RUN ##

links = []
anakthsh(links, 60)        # findAllPageLinks

# pipeRun(False, "Europe")

### Web-server 
#### Flask server running with ngrok

In [ ]:
import urllib.parse
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
  
app = Flask(__name__, template_folder="./Anakthsh/templates", static_folder="./Anakthsh/static")
run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("formApp.html")

@app.route("/elastic/<encodedQuery>")
def elasticSearch(encodedQuery):
    #trexe to piperun me to query
    query = urllib.parse.unquote(encodedQuery)
    elastic_results = pipeRun(False, query )
    return elastic_results

@app.route("/embedding/<encodedQuery>")
def embeddingSearch(encodedQuery):
    #trexe to piperun me to query
    query = urllib.parse.unquote(encodedQuery)    
    embedding_results = pipeRun(True, query )
    return embedding_results
app.run()

---
---
##Πιθανά προβλήματα

MaxRetryError: HTTPConnectionPool(host='localhost', port=60151): Max retries exceeded with url: /session/<> (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe7a900df90>: Failed to establish a new connection: [Errno 111] Connection refused'))

<b>Λύση: Ξαναφτιάχνω chrome-browser της selenium

In [ ]:
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

# browser = webdriver.Chrome('chromedriver',options=chrome_options)

WARNING - elasticsearch -  POST http://localhost:9200/_bulk?refresh=wait_for [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

<b>Λύση: Ξανατρέχω τον elasticsearch server

In [ ]:
# import os
# from subprocess import Popen,PIPE,STDOUT

# es_server = Popen(
#     ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)
# )
# !sleep 30

### Commented out tests

In [ ]:
# ##TESTS
# #downloadLinkInfo
# downloadLinkInfo(links)

# #getQA
# getQA(makeSoup("https://ec.europa.eu/commission/presscorner/detail/en/qanda_21_6621"), 3)

# #makeEmbedding
# soup = makeSoup("https://ec.europa.eu/commission/presscorner/detail/en/qanda_22_2322")
# findQuestions(soup)
# questions = soup.select("question")     # thelei get_text()
# questionsText = []
# for question in questions:
#   questionsText.append(question.get_text().strip())
# questions = questions.apply(lambda x: x.get_text.strip())
# makeEmbedding(questionsText, retrieverEmbedding)

# #isNotOrderListStyle
# for link in links[3]:
#   soup = makeSoup("https://ec.europa.eu/commission/presscorner/detail/en/qanda_21_6093")
#   section = soup.select("#inline-nav-1")[0]
#   print(isNotOrderListStyle(section))

# #isQuestion
# soupSection = makeSoup("https://ec.europa.eu/commission/presscorner/detail/en/qanda_22_2004").select("#inline-nav-1")[0]
# question = soupSection.select("p strong")[0]
# isQuestion(question)

# #findQuestions
# sectionTest = makeSoup("https://ec.europa.eu/commission/presscorner/detail/en/qanda_22_2004").select("#inline-nav-1")[0]
# print(isNotOrderListStyle(sectionTest))
# findQuestions(sectionTest)

# #getMetaInfo
# getMetaInfo(makeSoup("https://ec.europa.eu/commission/presscorner/detail/en/qanda_22_2322"))